In [1]:
import requests
from pymongo import MongoClient
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
#set up Mongo to store data 
client = MongoClient('localhost', 27017)

db = client['cap1_db']
col1 = db['col1']
col_people = db['col_people']
col_topd = db['col_topd']

In [30]:
%run scraping_functions.py


In [4]:
category_url = ['Data_scientists', 
                '20th-century_economists',
                '21st-century_economists', 
                '21st-century_chemists', 
                '20th-century_chemists',
                '21st-century_physicists',
                '20th-century_physicists',
                '21st-century_biologists',
                '20th-century_biologists',
                'Microbiologists',
                '21st-century_physicians',
                '20th-century_physicians']



In [5]:
for i in category_url: 
    # save html to doc
    html_to_collection(col1, i)
    #add list of names and links to doc
    people_to_collection(col1, i)
    #add list of subcategories to doc
    find_subcategories(col1, i)
    #grab list of names from the next page if there is one
    names_on_next_page(col1, i)

In [6]:
subcategory_url = set()
write_subcategories_to_set(col1, subcategory_url)

subcategory_old = set()
subcat_rounds = 0

while subcat_rounds < 2: 

    only_new_subcats = subcategory_url.difference(subcategory_old)
    
    for i in only_new_subcats: 
        # save html to doc
        html_to_collection(col1, i)
        #add list of names and links to doc
        people_to_collection(col1, i)
        #add list of subcategories to doc
        find_subcategories(col1, i)
        #grab list of names from the next page if there is one
        names_on_next_page(col1, i)
    
    
    subcategory_old.update(only_new_subcats)
    write_subcategories_to_set(col1, subcategory_url)
            
    subcat_rounds +=1


20th-century_Dutch_biologists: I have no subcategories
21st-century_New_Zealand_medical_doctors: I have no subcategories
20th-century_Chinese_physicians: I have no subcategories
21st-century_Moroccan_physicians: I have no subcategories
21st-century_Spanish_physicians: I have no subcategories
21st-century_Argentine_physicians: I have no subcategories
20th-century_Romanian_physicians: I have no subcategories
20th-century_Irish_medical_doctors: I have no subcategories
20th-century_French_physicians: I have no subcategories
20th-century_New_Zealand_medical_doctors: I have no subcategories
20th-century_Iranian_physicians: I have no subcategories
21st-century_Australian_medical_doctors: I have no subcategories
20th-century_Egyptian_physicians: I have no subcategories
21st-century_Dutch_biologists: I have no subcategories
21st-century_Indian_medical_doctors: I have no subcategories
20th-century_British_physicists: I have no subcategories
20th-century_Danish_physicians: I have no subcategories

20th-century_Indian_chemists: I have no subcategories
Palestinian_microbiologists: I have no subcategories
20th-century_French_botanists: I have no subcategories
Soviet_military_doctors: I have no subcategories
Soviet_women_physicists: I have no subcategories
Friedman_family: I have no subcategories
Soviet_biophysicists: I have no subcategories
Taxa_named_by_Thomas_Cavalier-Smith: I have no subcategories
Bloomsbury_Group_in_LGBT_history: I have no subcategories
20th-century_American_women_physicians: I have no subcategories
Robin_F%C3%A5hr%C3%A6us: I have no subcategories
Bacteriologists_by_nationality oh no i couldn't find people
Bacteriologists_by_nationality: something went wrong
Austrian_microbiologists: I have no subcategories
21st-century_British_botanists: I have no subcategories
New_Zealand_microbiologists: I have no subcategories
21st-century_German_botanists: I have no subcategories
Taxa_named_by_Stanislaw_Adamczewski: I have no subcategories
Jewish_bacteriologists: I have no

In [5]:
#check how many pages we have...
num_pages = 0
for x in col1.find():
    num_pages +=1

print(f"There are {num_pages} documents")

There are 280 documents


In [8]:
#perhaps could go ahead and remove categories that are less relevant
col1.delete_many({"page": {"$regex": 'Taxa?'}})
col1.delete_many({"page": {"$regex": 'Books?'}})
col1.delete_many({"page": {"$regex": 'Fictional?'}})
col1.delete_many({"page": {"$regex": 'Bloomsbury?'}})
col1.delete_many({"page": {"$regex": '\_Nazi?'}})
col1.delete_many({"page": 'Antonie_van_Leeuwenhoek'})
col1.delete_many({"page": 'Max_Planck'})
col1.delete_many({"page": 'Friedrich_Hayek'})
col1.delete_many({"page": {"$regex": '\Friedman?'}})
col1.delete_many({"page": 'Max_Planck'})
col1.delete_many({"page": 'Marie_Curie'})
col1.delete_many({"page": 'B._R._Ambedkar'})
col1.delete_many({"page": 'Richard_Feynman'})
col1.delete_many({"page": 'Werner_Heisenberg'})
col1.delete_many({"page": 'Virginia_Woolf'})

col1.delete_many({"page": 'Amartya_Sen'})
col1.delete_many({"page": 'Strachey_family'})
col1.delete_many({"page": 'Cultural_depictions_of_Marie_Curie'})
col1.delete_many({"page": 'Curie_family'})
col1.delete_many({"page": 'Robin_F%C3%A5hr%C3%A6us'})
col1.delete_many({"page": 'Manmohan_Singh'})
col1.delete_many({"page": 'E._M._Forster'})
col1.delete_many({"page": 'John_Maynard_Keynes'})



In [7]:
#grab all the page:

list_all_pages = []
for doc in col1.find():
    page = doc['page']
    list_all_pages.append(page)

#check how many pages we have...
len(list_all_pages)

280

In [8]:
progress = 0
for page in list_all_pages:
    progress += 1
    print(f"{progress}: {page}")
    people_html_to_collection(col1, page, col_people)

1: Data_scientists
2: 20th-century_economists
3: 21st-century_economists
4: 21st-century_chemists
5: 20th-century_chemists
6: 21st-century_physicists
7: 20th-century_physicists
8: 21st-century_biologists
9: 20th-century_biologists
10: Microbiologists
11: 21st-century_physicians
12: 20th-century_physicians
13: 20th-century_Dutch_biologists
14: 20th-century_British_medical_doctors
15: Women_microbiologists
16: 21st-century_New_Zealand_medical_doctors
17: 20th-century_Chinese_physicians
18: 21st-century_Moroccan_physicians
19: 21st-century_Spanish_physicians
20: 21st-century_women_physicians
21: 21st-century_Argentine_physicians
22: 20th-century_Romanian_physicians
23: 20th-century_Irish_medical_doctors
24: 20th-century_Indian_economists
25: 21st-century_botanists
26: 20th-century_French_physicians
27: 20th-century_New_Zealand_medical_doctors
28: 20th-century_Iranian_physicians
29: 20th-century_German_biologists
30: 21st-century_Australian_medical_doctors
31: 20th-century_Egyptian_physici

235: Corresponding_Members_of_the_USSR_Academy_of_Medical_Sciences
236: 20th-century_English_medical_doctors
237: 20th-century_Scottish_medical_doctors
238: Soviet_infectious_disease_physicians
239: 20th-century_Welsh_medical_doctors
240: Strachey_family
241: American_microbiologists
242: Ukrainian_microbiologists
243: Belgian_microbiologists
244: 21st-century_Indian_chemists
245: 21st-century_American_zoologists
246: South_African_microbiologists
247: Irish_microbiologists
248: Soviet_cardiologists
249: Argentine_microbiologists
250: Israeli_microbiologists
251: Finnish_microbiologists
252: Greek_microbiologists
253: Swiss_microbiologists
254: Soviet_endocrinologists
255: Iranian_microbiologists
256: Women_bacteriologists
257: Bulgarian_microbiologists
258: Jewish_microbiologists
259: Polish_microbiologists
260: Estonian_microbiologists
261: Soviet_women_physicians
262: Soviet_astrophysicists
263: 20th-century_American_botanists
264: 20th-century_Indian_zoologists
265: Ugandan_microbi

In [ ]:
col_people.delete_many({"page": 'Microbiologist'})
col_people.delete_many({"page": 'Protistologist'})
col_people.delete_many({"page": 'Protistology'})
col_people.delete_many({"page": 'Wikipedia:FAQ/Categorization'})
col_people.delete_many({"page": 'Alteryx'})
col_people.delete_many({"page": 'List of economic advisors to Donald Trump'})
col_people.delete_many({"page": {"$regex": 'Category?'}})




In [36]:
list_all_people = []
for doc in col_people.find():
    page = doc['page']
    list_all_people.append(page)



In [ ]:
# for person in list_all_people:
#     just_body_text(col_people, person)

In [22]:
len(list_all_people)

18312

In [37]:
len(list_all_people)

18310

In [32]:
progress = 0
hundred = 0
for person in list_all_people:
    progress +=1 
    count_gendered_words(col_people, person)
    if progress == 100:
        hundred += 1
        print(f"{hundred}: {person}: another 100" )
        progress = 0

Stoyan Alexandrov: another 100
Jozef Martin Paul van Brabant: another 100
Richard F. Ericson: another 100
Stephen Hymer: another 100
Alfonso López Pumarejo: another 100
Torsten Persson: another 100
Willem Somermeyer: another 100
Osip Yermansky: another 100
William Spriggs: another 100
Liu Xu (chemist): another 100
Mary Corner: another 100
Eugen Piwowarsky: another 100
Juan Ignacio Cirac Sasturain: another 100
Joachim Luther: another 100
Andreas Solberg Wahl: another 100
Pavel Cherenkov: another 100
Abram Ioffe: another 100
Gustav Naan: another 100
Edward Hutchinson Synge: another 100
Margaret Frame (biologist): another 100
Murat Aitkhozhin: another 100
Aleksandr Formozov: another 100
Zhores Medvedev: another 100
Walborg Thorsell: another 100
Adolf Winkelmann (physician): another 100
Carlos Chagas Filho: another 100
Tjipto Mangoenkoesoemo: another 100
Theodor Heinrich Schiebler: another 100
John Crighton Bramwell: another 100
Thomas Robinson Glynn: another 100
Frank Marsh (nephrologist)

In [43]:
num_ppl = 0 
for doc in col_people.find():
    num_ppl +=1
    
num_ppl

18212

In [44]:
df = pd.DataFrame(col_people.find())

In [45]:
df

,_id,page,field,html,body_text,doctorate,count_female_words,count_male_words,count_nonbinary_words,len_page,word_dict,years
0,5f9a37ac0cdf322c0cd01dd1,Amalia Fleming,20th-century_Greek_physicians,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",Greek physician bacteriologist Amalia Fleming...,0,44,5,0,2010,"{'greek': 24, 'physician': 3, '': 454, 'bacter...","[1912, 1986, 1912, 1986, 1974, 1912, 1914, 193..."
1,5f9a37ad0cdf322c0cd01dd2,Marietta Giannakou,20th-century_Greek_physicians,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",Greek politician Marietta GiannakouMember of t...,0,19,0,0,732,"{'greek': 9, 'politician': 2, 'marietta': 3, '...","[2009, 2014, 2004, 2007, 1990, 1991, 1951, 195..."
2,5f9a37ad0cdf322c0cd01dd3,George Higoumenakis,20th-century_Greek_physicians,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",Georgios George Higoumenakis 1895–27 Decemb...,0,0,11,0,391,"{'georgios': 1, '': 96, 'george': 4, 'higoumen...","[1895, 1983, 1924, 1927, 1940, 1964, 1967, 189..."
3,5f9a37ad0cdf322c0cd01dd4,Dionysios Ikkos,20th-century_Greek_physicians,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",This article is an orphan as no other article...,0,1,5,0,749,"{'this': 3, 'article': 2, 'is': 2, 'an': 2, 'o...","[2011, 1921, 1993, 1962, 1776, 1804, 1172, 104..."
4,5f9a37ad0cdf322c0cd01dd5,John Ioannidis,20th-century_Greek_physicians,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",American scientist John P A IoannidisBorn 1...,0,0,22,0,3471,"{'american': 2, 'scientist': 3, 'john': 22, 'p...","[1965, 1965, 1965, 2020, 2005, 1965, 1984, 199..."
...,...,...,...,...,...,...,...,...,...,...,...,...
18207,5f9aafa8be2b0f5fd05e42d1,Fritz Plato,20th-century_German_chemists,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",German chemist Fritz Plato 1858 – 1938 was a...,0,0,1,0,115,"{'german': 4, 'chemist': 3, 'fritz': 1, 'plato...",NaN
18208,5f9aafacbe2b0f5fd05e42d2,Friedrich Raschig,20th-century_German_chemists,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",Friedrich RaschigFriedrich Raschig 1897 Born...,0,0,9,0,724,"{'friedrich': 2, 'raschigfriedrich': 1, 'rasch...",NaN
18209,5f9aafb3be2b0f5fd05e42d3,Franz von Soxhlet,20th-century_German_chemists,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",German agricultural chemist Franz Ritter von...,0,0,6,0,773,"{'german': 3, 'agricultural': 3, 'chemist': 3,...",NaN
18210,5f9aafb6be2b0f5fd05e42d4,Andreas von Antropoff,20th-century_German_chemists,"b'<!DOCTYPE html>\n<html class=""client-nojs"" l...",Andreas von Antropoff born on August 16 1878...,0,0,23,0,1779,"{'andreas': 12, 'von': 20, 'antropoff': 28, ''...",NaN


In [46]:
df_tocsv = df[['page', 'field', 'count_female_words', 'count_male_words', 'count_nonbinary_words', 'doctorate', 'len_page']]


In [47]:
df_tocsv.to_csv('data/wiki_profile.csv')

In [245]:
subcat_dict = dict()
for i in category_url:
    doc = col1.find_one({'page': i })
    subcat_dict[i] = doc['subcat_link']


{'Data_scientists': None,
 '20th-century_economists': None,
 '21st-century_economists': None,
 '21st-century_chemists': None,
 '20th-century_chemists': None,
 '21st-century_physicists': None,
 '20th-century_physicists': None,
 '21st-century_biologists': None,
 '20th-century_biologists': None,
 'Microbiologists': None,
 '21st-century_physicians': None,
 '20th-century_physicians': None}

In [268]:
subcat_df = pd.Series(subcat_dict[category_url[0]])
for i in category_url[1:]:
    series_add = pd.Series(subcat_dict[i])
    subcat_df = pd.concat([subcat_df, series_add], axis=1)

In [276]:
subcat_df.columns = category_url
subcat_df.to_csv('data/subcategory1.csv')

In [295]:
subcat_dict2 = dict()
subcategory_category_url = []
for i in category_url: 
    doc = col1.find_one({'page': i})
    subcat_list = doc['subcat_link']
    for subcat in subcat_list:
        subcat = subcat.replace('/wiki/Category:', '')
        subcategory_category_url.append(subcat)
        subdoc = col1.find_one({'page': subcat })
        try:
            subcat_dict2[subcat] = subdoc['subcat_link']
        except:
            subcat_dict2[subcat] = None

In [299]:
subcat2_df = pd.Series(subcat_dict2[subcategory_category_url[0]])
for i in subcategory_category_url[1:]:
    series_add = pd.Series(subcat_dict2[i])
    subcat2_df = pd.concat([subcat2_df, series_add], axis=1)

subcat2_df.columns = subcategory_category_url


/Users/isabella/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.
/Users/isabella/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [301]:
subcat2_df.to_csv('data/subcategory2.csv')

In [345]:
test_col = db['test4']
r = requests.get('https://en.wikipedia.org/wiki/Phil_Torres')
test_col.insert_one({'page': 'Phil_Torres', 
                    'field': 'biologist',
                     'html': r.content})

In [346]:
just_body_text(test_col, 'Phil_Torres')